<a href="https://colab.research.google.com/github/PashaIanko/Kaggle.Restaurant-Revenue-Prediction/blob/feature_engineering/7_book_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is not a part of portfolio

This notebook contains messy code, because I practice through Aurelien Geron's book "Hands on Guide to Machine Learning", doing the exercises

# Imports

In [159]:
from google.colab import drive

import pandas as pd
import os

In [160]:
# PEP 8 
%%capture 
!pip install pycodestyle 
!pip install --index-url https://test.pypi.org/simple/ nbpep8 
from nbpep8.nbpep8 import pep8

In [161]:
%%capture

git_download_path = \
'https://raw.githubusercontent.com/PashaIanko/Kaggle.Restaurant-Revenue-Prediction/main/'

def download_files(dict_):
    for file, url in dict_.items():
        !wget -O {file} {url} {file}


files = [
    'path_manager.py', 
    'sklearn_utils.py',
    'sklearn_transformers.py'
]

dict_ = {f: git_download_path + f for f in files}
download_files(dict_)
    

pep8(_ih)

In [162]:
import path_manager
import sklearn_utils
import sklearn_transformers
import importlib


def reload(modules):
    for m in modules:
        importlib.reload(m)


modules = [
    path_manager,
    sklearn_utils,
    sklearn_transformers,
]

reload(modules)

from path_manager import PathManager

# Set up paths

In [163]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [164]:
ROOT_PATH = '/content/gdrive/MyDrive/'
COMPETITION_PATH = ROOT_PATH + 'ML/Competitions/5.RestaurantRevenue/'

PREPROCESSING_TRIAL = 2
MODELS_TRIAL = 2

In [165]:
manager = PathManager(
    competition_path = COMPETITION_PATH,
    preprocessing_trial = PREPROCESSING_TRIAL,
    models_trial = MODELS_TRIAL
)

In [166]:
df_train = pd.read_csv(
    os.path.join(
        manager.data_root_path,
        'train.csv'
    )
)

pep8(_ih)

In [167]:
df_train.head()

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
0,0,07/17/1999,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,...,3.0,5,3,4,5,5,4,3,4,5653753.0
1,1,02/14/2008,Ankara,Big Cities,FC,4,5.0,4.0,4.0,1,...,3.0,0,0,0,0,0,0,0,0,6923131.0
2,2,03/09/2013,Diyarbakır,Other,IL,2,4.0,2.0,5.0,2,...,3.0,0,0,0,0,0,0,0,0,2055379.0
3,3,02/02/2012,Tokat,Other,IL,6,4.5,6.0,6.0,4,...,7.5,25,12,10,6,18,12,12,6,2675511.0
4,4,05/09/2009,Gaziantep,Other,IL,3,4.0,3.0,4.0,2,...,3.0,5,1,3,2,3,4,3,3,4316715.0


# Preprocessing pipeline

In [168]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn_transformers import ColumnDropper

In [169]:
num_pipeline = Pipeline(
    [
        ('impute', SimpleImputer()),
        ('scale', StandardScaler())
    ]
)

cat_pipeline = Pipeline(
    [
        ('drop', ColumnDropper(['Open Date'])),
        ('one-hot', OneHotEncoder())
    ]
)

In [170]:
import numpy as np
np.unique(df_train.dtypes)

array([dtype('int64'), dtype('float64'), dtype('O')], dtype=object)

In [171]:
cat_cols = df_train.columns[df_train.dtypes == 'object']
num_cols = df_train.columns[df_train.dtypes.isin(['float64', 'int64'])]

In [172]:
num_cols

Index(['Id', 'P1', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P14',
       'P15', 'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24',
       'P25', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35', 'P36', 'P37'],
      dtype='object')

In [173]:
cat_cols

Index(['Open Date', 'City', 'City Group', 'Type'], dtype='object')

In [175]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    [
        ('numeric', num_pipeline, num_cols),
        ('categorical', cat_pipeline, cat_cols)
    ]
)

In [176]:
Y = df_train['revenue']
df_train.drop('revenue', inplace=True, axis='columns')

In [177]:
train_transformed = preprocessor.fit_transform(
    df_train
)

# Exercise 1

Try a Support Vector Machine regressor (sklearn.svm.SVR), with various hyperparameters
such as kernel="linear" (with various values for the C hyperparameter)
or kernel="rbf" (with various values for the C and gamma
hyperparameters). Don’t worry about what

In [178]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

In [179]:
model = SVR()

gs_result = GridSearchCV(
    estimator = model,
    param_grid = [
        {
            'kernel': ['linear'],
            'C': [0.001, 0.01, 0.1, 0.3, 0.6, 0.9]
        },
        {
            'kernel': ['rbf'],
            'C': [0.001, 0.01, 0.1, 0.3, 0.6, 0.9]
        }
    ],
    cv=5,
    scoring='neg_mean_squared_error',
    refit=True
)

In [180]:
gs_result.fit(
    X=train_transformed,
    y=Y
)

GridSearchCV(cv=5, estimator=SVR(),
             param_grid=[{'C': [0.001, 0.01, 0.1, 0.3, 0.6, 0.9],
                          'kernel': ['linear']},
                         {'C': [0.001, 0.01, 0.1, 0.3, 0.6, 0.9],
                          'kernel': ['rbf']}],
             scoring='neg_mean_squared_error')

In [181]:
gs_result.best_score_

-6883898183070.784

# Exercise 2

Replace GridSearchCV with RandomizedSearchCV

In [182]:
df_train.head(1)

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37
0,0,07/17/1999,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,...,2.0,3.0,5,3,4,5,5,4,3,4


In [183]:
df_train['City Group'].unique()

array(['Big Cities', 'Other'], dtype=object)

In [190]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder


num_pipeline = Pipeline(
    [
        ('impute', SimpleImputer(strategy='mean')),
        ('scale', MinMaxScaler())
    ]
)

cat_pipeline = Pipeline(
    [
        ('drop', ColumnDropper(['Open Date'])),
        ('ordinal', OrdinalEncoder()),
    ]
)

pep8(_ih)

In [191]:
num_cols = df_train.columns[df_train.dtypes.isin(['int64', 'float64'])]
cat_cols = df_train.columns[df_train.dtypes == 'object']

In [192]:
num_cols

Index(['Id', 'P1', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P14',
       'P15', 'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24',
       'P25', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35', 'P36', 'P37'],
      dtype='object')

In [193]:
cat_cols

Index(['Open Date', 'City', 'City Group', 'Type'], dtype='object')

In [194]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    [
        ('numeric', num_pipeline, num_cols),
        ('categorical', cat_pipeline, cat_cols)
    ]
)

In [195]:
df_train_transformed = preprocessor.fit_transform(df_train)

In [196]:
df_train_transformed.shape

(137, 33)

In [197]:
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV

In [199]:
rand_search = RandomizedSearchCV(
    SVR(),
    [
        {
            'kernel': ['linear'],
            'C': [0.1 * i for i in range(1, 10)]
        },
        {
            'kernel': ['rbf'],
            'C': [0.1 * i for i in range(1, 10)]
        }
    ],
    n_iter = 8,
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42,
    return_train_score=True
)

In [201]:
rand_res = rand_search.fit(X=df_train_transformed, y=Y)

In [203]:
rand_res.best_estimator_

SVR(C=0.9, kernel='linear')

# Exercise 3

Try adding a transformer in the preparation pipeline to select only the most
important attributes.

In [ ]:
df_train.head(1)

In [297]:
from sklearn.base import TransformerMixin, BaseEstimator

class ColumnKeeper(TransformerMixin, BaseEstimator):
    def __init__(self, columns_to_keep):
        super().__init__()
        self.columns_to_keep = columns_to_keep
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        print('AAA', type(X))
        print(X.columns)
        return X.loc[:, self.columns_to_keep]

In [299]:
c = ColumnKeeper(['Open Date'])
res = c.fit_transform(df_train)
res.head()

AAA <class 'pandas.core.frame.DataFrame'>
Index(['Id', 'Open Date', 'City', 'City Group', 'Type', 'P1', 'P2', 'P3', 'P4',
       'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15',
       'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25',
       'P26', 'P27', 'P28', 'P29', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35',
       'P36', 'P37'],
      dtype='object')


,Open Date
0,07/17/1999
1,02/14/2008
2,03/09/2013
3,02/02/2012
4,05/09/2009


In [300]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn_transformers import ColumnDropper


keep_pipeline = Pipeline(
    [
        ('keep', ColumnKeeper(['Open Date', 'Type', 'P2']))
    ]
)

num_pipeline = Pipeline(
    [
        ('impute', SimpleImputer()),
        ('scale', StandardScaler()),
        
    ]
)

cat_pipeline = Pipeline(
    [
        ('drop', ColumnDropper(['Open Date'])),
        ('encode', OrdinalEncoder()),
    ]
)

In [317]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    [
        ('keep', keep_pipeline, list(df_train.columns)),
        ('num', num_pipeline, num_cols),
        ('cat', cat_pipeline, cat_cols)
    ]
)

In [318]:
df_train_proc = preprocessor.fit_transform(df_train)

In [319]:
df_train_proc.shape

(137, 33)

In [320]:
df_train_proc[0, :]

array([-1.71945391e+00, -5.03441458e-03, -6.05649194e-03, -6.38470683e-01,
       -1.85000681e-01, -6.22800667e-01, -2.43563924e-01, -2.65672281e-01,
       -1.38027313e-01, -1.54696055e-01, -1.52984995e-01,  2.56553440e-01,
        1.67175352e-02,  4.76214771e-01,  6.25940687e-01,  1.69933624e-02,
       -1.48179102e-01, -6.21024720e-01,  6.30997096e-01, -9.31902780e-02,
       -1.62157449e-01, -9.95958051e-02,  4.11513207e-01,  3.02462963e-01,
        2.82950354e-01,  2.28165658e+00,  4.87922157e-01,  5.75634085e-01,
        1.89821043e-01,  1.61595083e+00,  3.10000000e+01,  0.00000000e+00,
        2.00000000e+00])

In [316]:
df_train_proc[0, :]

array(['07/17/1999', 'IL', 5.0, -1.7194539072421902,
       -0.005034414575988356, -0.006056491935424382, -0.6384706830556539,
       -0.18500068055518223, -0.622800667328129, -0.24356392382330014,
       -0.2656722812155785, -0.1380273129398537, -0.15469605534873526,
       -0.15298499464541934, 0.2565534399370158, 0.016717535181142894,
       0.47621477100318504, 0.6259406873659143, 0.016993362421990437,
       -0.14817910163884068, -0.6210247198234383, 0.6309970960497727,
       -0.093190278046297, -0.162157448839831, -0.09959580513671422,
       0.41151320679761516, 0.3024629630523616, 0.2829503544940959,
       2.281656583714585, 0.4879221566329955, 0.5756340846579672,
       0.18982104327431898, 1.615950825715222, 31.0, 0.0, 2.0],
      dtype=object)

# Exercise 4

In [364]:
num_pipeline = Pipeline(
    [
        ('impute', SimpleImputer()),
        ('scale', MinMaxScaler())
    ]
)

cat_pipeline = Pipeline(
    [
        ('drop', ColumnDropper(['Open Date', 'City', 'Type'])),
        ('Encode', OrdinalEncoder())
    ]
)

In [365]:
df_train.columns

Index(['Id', 'Open Date', 'City', 'City Group', 'Type', 'P1', 'P2', 'P3', 'P4',
       'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15',
       'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25',
       'P26', 'P27', 'P28', 'P29', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35',
       'P36', 'P37'],
      dtype='object')

In [366]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    [
        
        ('numeric', num_pipeline, num_cols),
        ('cat', cat_pipeline, cat_cols)
    ]
)

In [367]:
df_processed = preprocessor.fit_transform(df_train)

In [358]:
pd.DataFrame(df_processed[:5, :]).head()

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,0.000000,0.272727,0.142857,0.111111,0.444444,0.333333,0.166667,0.166667,0.222222,0.375,...,0.200000,0.16,0.833333,0.208333,0.266667,0.15,0.500,31.0,0.0,2.0
1,0.007353,0.272727,0.000000,0.111111,0.444444,0.444444,0.166667,0.166667,0.000000,0.375,...,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000,3.0,0.0,1.0
2,0.014706,0.090909,0.142857,0.222222,0.444444,0.444444,0.166667,0.166667,0.111111,0.375,...,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000,10.0,1.0,2.0
3,0.022059,0.454545,0.428571,0.333333,1.000000,0.777778,1.000000,1.000000,0.777778,1.000,...,0.800000,0.40,1.000000,0.750000,0.800000,0.60,0.750,28.0,1.0,2.0
4,0.029412,0.181818,0.142857,0.111111,0.444444,0.444444,0.166667,0.166667,0.111111,0.375,...,0.066667,0.12,0.333333,0.125000,0.266667,0.15,0.375,14.0,1.0,2.0


In [384]:
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor

full_prediction_pipe = Pipeline(
    [
        ('preproc', preprocessor),
        ('predict', RandomForestRegressor(n_estimators=3))
    ]
)

In [385]:
df_train.shape

(137, 42)

In [388]:
preds = full_prediction_pipe.fit(X=df_train.loc[:99, :],y=Y[:100])

# Exercise 5

Automatically check some data preparation options, using GridSearchCV

In [389]:
df_train.head(1)

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37
0,0,07/17/1999,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,...,2.0,3.0,5,3,4,5,5,4,3,4


In [426]:
from sklearn.pipeline import Pipeline

num_pipeline = Pipeline(
    [
        ('impute', SimpleImputer()),
        ('scale', StandardScaler())
    ]
)

cat_pipeline = Pipeline(
    [
        ('drop', ColumnDropper(['Open Date', 'City', 'Type'])),
        ('encode', OrdinalEncoder())
    ]
)

In [427]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    [
        ('num', num_pipeline, num_cols),
        ('cat', cat_pipeline, cat_cols)
    ]
)

In [428]:
from sklearn.ensemble import RandomForestRegressor

final_pipe = Pipeline(
    [
        ('preproc', preprocessor),
        ('model', RandomForestRegressor())
    ]
)

In [418]:
final_pipe.fit(df_train, Y)

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   StandardScaler())]),
                                                  ['Id', 'P1', 'P5', 'P6', 'P7',
                                                   'P8', 'P9', 'P10', 'P11',
                                                   'P12', 'P14', 'P15', 'P16',
                                                   'P17', 'P18', 'P19', 'P20',
                                                   'P21', 'P22', 'P23', 'P24',
                                                   'P25', 'P30', 'P31', 'P32',
                                                   'P33', 'P34', 'P35', 'P36',
                                    

In [429]:
from sklearn.model_selection import GridSearchCV

gs_estimator = GridSearchCV(
    final_pipe,
    {
        'model__n_estimators': [500, 150, 5, 100, 300],
        'preproc__num__impute__strategy': ['mean', 'median']
    },
    cv=5
)

gs_estimator.fit(df_train, Y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preproc',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('impute',
                                                                                          SimpleImputer()),
                                                                                         ('scale',
                                                                                          StandardScaler())]),
                                                                         ['Id',
                                                                          'P1',
                                                                          'P5',
                                                                          'P6',
                                                                          'P7',
                                

In [430]:
gs_estimator.best_params_

{'model__n_estimators': 300, 'preproc__num__impute__strategy': 'mean'}